In [14]:
import numpy as np
import pandas as pd
import os

# Define path to the data and other constants
data_path = '/home/ximo/Escritorio/ProyectoTFG/Muse'
preprocessed_path = '/home/ximo/Escritorio/ProyectoTFG/MusePreprocessed'
subjects = range(1, 31)
repetitions = ['1', '2']
minutes = ['1', '2', '3']
experiment_types = ['m', 'l', 'c', 'e']
excluded_columns = ['TimeStamp', 'Battery', 'HeadBandOn', 'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Accelerometer_X', 'Accelerometer_Y', 'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z', 'Mellow', 'Concentration', 'Elements']

def load_and_preprocess_data(file_path):
    # Carga de datos
    data = pd.read_csv(file_path, dtype=str, low_memory=False, on_bad_lines='skip')
    
    # Asegurarse de que 'Elements' y las columnas excluidas están en el DataFrame
    existing_columns = data.columns.tolist()
    if 'Elements' in existing_columns:
        # Filtrar filas donde 'Elements' no está vacío (es decir, mantener solo las filas donde 'Elements' está vacío)
        data = data[~data['Elements'].isin(['/muse/elements/blink', '/muse/elements/jaw_clench'])]
    
    # Eliminar columnas excluidas solo si están presentes
    columns_to_drop = [col for col in excluded_columns if col in existing_columns]
    data.drop(columns=columns_to_drop, inplace=True)
    
    # Convertir todas las columnas a numérico, reemplazando errores con NaN
    data = data.apply(pd.to_numeric, errors='coerce')
    
    # Reemplazar NaN, inf, -inf con 0
    data.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

    missing_rows = 15360 - len(data)
    if missing_rows > 0:
        if not data.empty:
            # Encuentra la última fila no vacía
            last_valid_row = data[(data.T != 0).any()].iloc[-1] if not data[(data.T != 0).any()].empty else pd.Series([0]*len(data.columns), index=data.columns)
            # Replicar la última fila válida las veces que haga falta
            additional_rows = pd.DataFrame([last_valid_row] * missing_rows)
        else:
            # Si el dataframe está completamente vacío, crea filas llenas de ceros
            raise ValueError("dataframe está completamente vacío.")
        
        # Concatenar las filas adicionales al DataFrame original
        data = pd.concat([data, additional_rows], ignore_index=True)


    if(missing_rows<0):
        # Eliminar filas que son completamente cero, pero asegurando al menos 15360 filas
        all_zero_rows = data[(data.T == 0).all()].index
        if len(data) - len(all_zero_rows) >= 15360:
            rows_to_remove = len(all_zero_rows) - (len(data) - 15360)
            data.drop(all_zero_rows[:rows_to_remove], inplace=True)
        else:
            rows_to_keep = 15360 - (len(data) - len(all_zero_rows))
            data.drop(all_zero_rows[rows_to_keep:], inplace=True)
    
    return data



def compute_global_min_max(data_path, subjects, repetitions, experiment_types, minutes, excluded_columns):
    min_vals, max_vals = None, None

    for subject in subjects:
        for repetition in repetitions:
            for exp_type in experiment_types:
                for minute in minutes:
                    file_path = os.path.join(data_path, f'{subject}_{repetition}_{exp_type}{minute}.csv')
                    try:
                        data = pd.read_csv(file_path, dtype=str, low_memory=False, on_bad_lines='skip')
                        data.drop(columns=excluded_columns, inplace=True)
                        data = data.apply(pd.to_numeric, errors='coerce')

                        # Limpieza inicial de datos
                        data.replace([np.inf, -np.inf], np.nan, inplace=True)
                        current_min = data.min(skipna=True)
                        current_max = data.max(skipna=True)

                        # Comprueba si hay datos válidos antes de proceder
                        if current_min.notna().any() or current_max.notna().any():
                            if min_vals is None:
                                min_vals = current_min
                                max_vals = current_max
                            else:
                                # Asegúrate de que solo se actualizan los índices donde current_min/max es no-NaN
                                valid_min_indices = current_min.notna()
                                valid_max_indices = current_max.notna()
                                min_vals[valid_min_indices] = np.minimum(min_vals[valid_min_indices], current_min[valid_min_indices])
                                max_vals[valid_max_indices] = np.maximum(max_vals[valid_max_indices], current_max[valid_max_indices])
                    except Exception as e:
                        print(f"Error processing file {file_path}: {e}")

    # Verificar si al menos hay un mínimo y máximo global válido
    if min_vals is None or max_vals is None or min_vals.isna().all() or max_vals.isna().all():
        raise ValueError("No valid data found for one or more columns across all datasets.")

    return min_vals, max_vals



def normalize_and_save_data(data, min_vals, max_vals, save_path):
    # Normalize data
    diff = max_vals - min_vals
    diff.replace(0, 1, inplace=True)  # Prevent division by zero
    normalized_data = (data - min_vals) / diff
    
    # Save the normalized data
    normalized_data.to_csv(save_path, index=False)

# Ensure the preprocessed directory exists
os.makedirs(preprocessed_path, exist_ok=True)

# Compute global min and max
min_vals, max_vals = compute_global_min_max(data_path, subjects, repetitions, experiment_types, minutes, excluded_columns)
print(min_vals)
print("\n")
print(max_vals)
print("\n")

for subject in subjects:
    for repetition in repetitions:
        for exp_type in experiment_types:
            for minute in minutes:
                file_name = f'{subject}_{repetition}_{exp_type}{minute}.csv'
                file_path = os.path.join(data_path, file_name)
                save_path = os.path.join(preprocessed_path, file_name)
                
                if os.path.exists(file_path):
                    data = load_and_preprocess_data(file_path)
                    normalize_and_save_data(data, min_vals, max_vals, save_path)


Delta_TP9     -0.979058
Delta_AF7     -1.144020
Delta_AF8    -91.000000
Delta_TP10    -4.155029
Theta_TP9     -0.872132
Theta_AF7     -1.418440
Theta_AF8     -1.130179
Theta_TP10    -4.433084
Alpha_TP9     -0.359909
Alpha_AF7     -0.688447
Alpha_AF8     -0.596675
Alpha_TP10    -4.169735
Beta_TP9      -0.387568
Beta_AF7      -0.794948
Beta_AF8      -0.629749
Beta_TP10     -3.761396
Gamma_TP9     -0.766902
Gamma_AF7     -1.060367
Gamma_AF8     -1.064831
Gamma_TP10    -3.690269
RAW_TP9        0.000000
RAW_AF7        0.000000
RAW_AF8        0.000000
RAW_TP10       0.000000
AUX_RIGHT      0.000000
dtype: float64


Delta_TP9     4.179489e+00
Delta_AF7     3.625859e+00
Delta_AF8     2.840539e+00
Delta_TP10    4.274526e+00
Theta_TP9     3.207157e+00
Theta_AF7     2.672567e+00
Theta_AF8     9.040000e+02
Theta_TP10    1.414000e+03
Alpha_TP9     8.960000e+02
Alpha_AF7     2.889222e+00
Alpha_AF8     2.228410e+00
Alpha_TP10    3.041323e+00
Beta_TP9      2.697978e+00
Beta_AF7      5.990000e+02
Beta_

In [18]:
import pandas as pd
import os

def trim_csv_files(directory, max_rows):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Asegura procesar solo archivos CSV
            filepath = os.path.join(directory, filename)
            # Carga el archivo CSV con Pandas
            df = pd.read_csv(filepath)
            # Trunca el dataframe a las primeras max_rows filas
            trimmed_df = df.head(max_rows)
            # Sobrescribe el archivo original con la versión truncada
            trimmed_df.to_csv(filepath, index=False)
            print(f"Archivo {filename} truncado a {max_rows} filas.")

# Especifica el directorio donde se encuentran tus archivos CSV
directory_path = '/home/ximo/Escritorio/ProyectoTFG/MusePreprocessed'
# Llama a la función con el directorio y el número de filas deseadas
trim_csv_files(directory_path, 15360)


Archivo 21_2_e3.csv truncado a 15360 filas.
Archivo 19_2_c1.csv truncado a 15360 filas.
Archivo 26_2_l2.csv truncado a 15360 filas.
Archivo 3_2_l1.csv truncado a 15360 filas.
Archivo 24_2_m2.csv truncado a 15360 filas.
Archivo 3_1_e1.csv truncado a 15360 filas.
Archivo 29_1_c3.csv truncado a 15360 filas.
Archivo 4_1_e3.csv truncado a 15360 filas.
Archivo 15_2_e1.csv truncado a 15360 filas.
Archivo 12_1_c1.csv truncado a 15360 filas.
Archivo 2_1_m3.csv truncado a 15360 filas.
Archivo 28_1_m3.csv truncado a 15360 filas.
Archivo 8_1_l1.csv truncado a 15360 filas.
Archivo 9_2_m3.csv truncado a 15360 filas.
Archivo 5_1_e3.csv truncado a 15360 filas.
Archivo 12_2_c2.csv truncado a 15360 filas.
Archivo 30_1_m2.csv truncado a 15360 filas.
Archivo 4_2_e1.csv truncado a 15360 filas.
Archivo 11_2_e2.csv truncado a 15360 filas.
Archivo 13_2_m3.csv truncado a 15360 filas.
Archivo 8_1_m1.csv truncado a 15360 filas.
Archivo 16_2_m1.csv truncado a 15360 filas.
Archivo 11_1_l3.csv truncado a 15360 fila

In [19]:
import os
import pandas as pd

def count_rows_in_csv_files(directory):
    # List all files in the given directory
    files = os.listdir(directory)
    for file in files:
        # Check if the file is a CSV
        if file.endswith('.csv'):
            file_path = os.path.join(directory, file)
            # Read the CSV file
            data = pd.read_csv(file_path)
            # Get the number of rows
            num_rows = len(data)
            # Print the file name and the number of rows
            print(f'File: {file} - Rows: {num_rows}')

# Path to the directory containing CSV files
directory_path = '/home/ximo/Escritorio/ProyectoTFG/MusePreprocessed'

# Call the function with the path to your directory
count_rows_in_csv_files(directory_path)


File: 21_2_e3.csv - Rows: 15360
File: 19_2_c1.csv - Rows: 15360
File: 26_2_l2.csv - Rows: 15360
File: 3_2_l1.csv - Rows: 15360
File: 24_2_m2.csv - Rows: 15360
File: 3_1_e1.csv - Rows: 15360
File: 29_1_c3.csv - Rows: 15360
File: 4_1_e3.csv - Rows: 15360
File: 15_2_e1.csv - Rows: 15360
File: 12_1_c1.csv - Rows: 15360
File: 2_1_m3.csv - Rows: 15360
File: 28_1_m3.csv - Rows: 15360
File: 8_1_l1.csv - Rows: 15360
File: 9_2_m3.csv - Rows: 15360
File: 5_1_e3.csv - Rows: 15360
File: 12_2_c2.csv - Rows: 15360
File: 30_1_m2.csv - Rows: 15360
File: 4_2_e1.csv - Rows: 15360
File: 11_2_e2.csv - Rows: 15360
File: 13_2_m3.csv - Rows: 15360
File: 8_1_m1.csv - Rows: 15360
File: 16_2_m1.csv - Rows: 15360
File: 11_1_l3.csv - Rows: 15360
File: 15_2_m3.csv - Rows: 15360
File: 18_1_e1.csv - Rows: 15360
File: 21_2_c1.csv - Rows: 15360
File: 29_1_m2.csv - Rows: 15360
File: 11_1_c2.csv - Rows: 15360
File: 15_1_c1.csv - Rows: 15360
File: 25_1_c3.csv - Rows: 15360
File: 8_1_m2.csv - Rows: 15360
File: 3_1_e2.csv -

In [20]:
import csv
import os

def check_empty_columns_in_csv(directory):
    # Recorre todos los archivos en el directorio especificado
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Asegúrate de que estás revisando solo archivos CSV
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', newline='') as file:
                reader = csv.reader(file)
                for row_index, row in enumerate(reader, start=1):
                    # Revisa si alguna 'celda' en la fila está vacía (dos comas seguidas o comienzo/final de línea con coma)
                    if any(len(column) == 0 for column in row) or ('' in row):
                        print(f"Archivo: {filename}, Línea: {row_index}, Contenido: {row}")

# Especifica el directorio donde se encuentran tus archivos CSV
directory_path = '/home/ximo/Escritorio/ProyectoTFG/MusePreprocessed'
check_empty_columns_in_csv(directory_path)


In [1]:
import pandas as pd
import os

# Ruta de la carpeta que contiene los archivos CSV
carpeta = '/home/ximo/Escritorio/ProyectoTFG/MusePreprocessed'

# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

# Iterar sobre cada archivo CSV
for archivo in archivos_csv:
    ruta_archivo = os.path.join(carpeta, archivo)
    
    # Cargar el archivo CSV en un DataFrame de pandas
    df = pd.read_csv(ruta_archivo)
    
    # Seleccionar solo las columnas que contienen 'RAW_'
    columnas_raw = [col for col in df.columns if 'RAW_' in col]
    
    # Crear un nuevo DataFrame solo con las columnas 'RAW_'
    df_raw = df[columnas_raw]
    
    # Guardar el nuevo DataFrame en el mismo archivo CSV sobrescribiendo el original
    df_raw.to_csv(ruta_archivo, index=False)
    
    print(f"Archivo '{archivo}' procesado. Ahora solo contiene las columnas 'RAW_'.")

print("Proceso completado.")


Archivo '21_2_e3.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '19_2_c1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '26_2_l2.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '3_2_l1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '24_2_m2.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '3_1_e1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '29_1_c3.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '4_1_e3.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '15_2_e1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '12_1_c1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '2_1_m3.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '28_1_m3.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '8_1_l1.csv' procesado. Ahora solo contiene las columnas 'RAW_'.
Archivo '9_2_m3.csv' procesado. Ahora solo 

In [4]:
import pandas as pd
import os

# Ruta de la carpeta que contiene los archivos CSV
carpeta = '/home/ximo/Escritorio/ProyectoTFG/featuresExtended'

# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

# Iterar sobre cada archivo CSV
for archivo in archivos_csv:
    ruta_archivo = os.path.join(carpeta, archivo)
    
    # Cargar el archivo CSV en un DataFrame de pandas
    df = pd.read_csv(ruta_archivo)
    
    # Seleccionar solo las columnas que contienen 'RAW_' o 'RIGHT'
    columnas_raw = [col for col in df.columns if 'RAW_' in col or 'RIGHT' in col]
    
    # Crear un nuevo DataFrame solo con las columnas que no contienen 'RAW_' o 'RIGHT'
    df_raw = df.drop(columns=columnas_raw)
    
    # Guardar el nuevo DataFrame en el mismo archivo CSV sobrescribiendo el original
    df_raw.to_csv(ruta_archivo, index=False)
    
    print(f"Archivo '{archivo}' procesado. Ahora solo contiene las columnas no redundantes.")

print("Proceso completado.")



Archivo 'features_1.csv' procesado. Ahora solo contiene las columnas no redundantes.
Archivo 'features_4.csv' procesado. Ahora solo contiene las columnas no redundantes.
Archivo 'features_3.csv' procesado. Ahora solo contiene las columnas no redundantes.
Archivo 'features_2.csv' procesado. Ahora solo contiene las columnas no redundantes.
Proceso completado.
